In [4]:
class DrawTrayectories:
    def __init__(self,ruta_de_video,video,nombre_de_carpeta):
        import os
        self.ruta_de_video = ruta_de_video
        self.video = video
        self.nombre_de_carpeta = nombre_de_carpeta
        self.path_originales = os.getcwd()+'/'+ self.nombre_de_carpeta + '/'
        
    def SacarTrayectorias(self):
        import subprocess
        subprocess.call(['bash', 'sacar_trayectorias.sh', self.video, self.ruta_de_video])
    
    def CargarTrayectoriasEnDataFrame(self):
        import pandas as pd
        Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
        Cabecera = pd.read_csv('cabecera', sep=' ', header=None)
        del Trayectories[34]
        return Trayectories, Cabecera
    
    def FiltrarTrayectorias(self, Trayectories, b):
        Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=b]
        Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=b]
        return Trayectories
        
    def InfoDeCabecera(self, Trayectories, Cabecera):
        Total_de_trayectorias = len(Trayectories[0])
        y = Cabecera[1].values[0]
        x = Cabecera[2].values[0]
        cantidad_de_fotogramas = Cabecera[3].values[0]
        print('Total de trayectorias: ' + str(Total_de_trayectorias), 'Alto: ' + str(y), 'Ancho: ' + str(x), 'Cantidad de fotogramas: ' + str(cantidad_de_fotogramas))
        return Total_de_trayectorias, cantidad_de_fotogramas, y, x
    
    
    def MatrizDeTrayectorias(self, dataframe, Total_de_trayectorias, Cantidad_de_fotogramas):
        import numpy as np
        matriz = np.zeros((Total_de_trayectorias, Cantidad_de_fotogramas + 1, 2))
        matriz[:,:] = -100
        data = dataframe.values
        matriz[:,Cantidad_de_fotogramas,0] = data[:,0]
        matriz[:,Cantidad_de_fotogramas,1] = data[:,1]
        for i in range(Total_de_trayectorias):
            tray = data[i,:].reshape((34,1))
            j = 0
            while j <= 32:
                if j == 0 or j == 1:
                    inicio = int(tray[0])-1
                else:

                    matriz[i,inicio,0] = tray[j]
                    matriz[i,inicio,1] = tray[j+1]
                    inicio = inicio + 1
                j = j + 2

        return matriz    
    
    
    def SacarFotogramas(self):
        import subprocess
        subprocess.call(['bash', 'sacar_fotogramas.sh', self.nombre_de_carpeta, self.video, self.ruta_de_video])
        
    def SortDir(self, path):
        import os
        import numpy as np
        frames = os.listdir(path)

        indices=[]

        for i in frames:
            indices.append(int(i[10:13]))


        sort_indices = np.sort(indices)
        sort_frames = []       
        for i in sort_indices:
                if i<10:
                    sort_frames.append("fotograma-00"+str(i)+".png") 
                elif i<100:  
                     sort_frames.append("fotograma-0"+str(i)+".png")
                else:
                     sort_frames.append("fotograma-"+str(i)+".png")

        return sort_frames
    
    
    
    def Dibujar(self, Total_de_trayectorias, cantidad_de_fotogramas, matriz_tray):
        import cv2
        frames_originales = self.SortDir(self.path_originales)
        for i,frame in enumerate(frames_originales):
            img = cv2.imread(self.path_originales + frame)
            if i+1 != len(frames_originales):
                for j in range(Total_de_trayectorias):
                    if(i+1>=matriz_tray[j,cantidad_de_fotogramas,0] and i+1<=matriz_tray[j,cantidad_de_fotogramas,1]):
                        #print(j)
                        tray = matriz_tray[j,:i+2,:]
                        tray_mask = tray != -100
                        tray = tray[tray_mask]
                        w = tray.shape
                        w = int(w[0]/2)
                        tray = tray.reshape((w,2))
                        k = 0
                        intencidad = 0
                        while k <= w-2:
                            if k == w-2:
                                img = cv2.line(img,(int(tray[k,1]),int(tray[k,0])),(int(tray[k+1,1]),int(tray[k+1,0])),(0,0,255),1)
                                k = k + 1
                            else:
                                intencidad = intencidad + 255*0.1
                                #print(tray[k,1],tray[k,0],tray[k+1,1],tray[k+1,0])
                                img = cv2.line(img,(int(tray[k,1]),int(tray[k,0])),(int(tray[k+1,1]),int(tray[k+1,0])),(0,intencidad,0),1)
                                k = k + 1
            cv2.imwrite(self.path_originales + frame , img)
            
            
    def ArmarVideo(self):
        import subprocess
        subprocess.call(['bash', 'armar_video.sh', self.video[:6] + '_con_trayectorias.mp4', self.nombre_de_carpeta,'25'])
        subprocess.call(['rm','-r',self.nombre_de_carpeta])
    
    def Run(self,b):
        self.SacarTrayectorias()
        Trayectories, Cabecera = self.CargarTrayectoriasEnDataFrame()
        Trayectories = self.FiltrarTrayectorias(Trayectories, b)
        Total_de_trayectorias, Cantidad_de_fotogramas, y, x = self.InfoDeCabecera(Trayectories, Cabecera)
        Matriz_tray = self.MatrizDeTrayectorias(Trayectories, Total_de_trayectorias, Cantidad_de_fotogramas)
        self.SacarFotogramas()
        self.Dibujar(Total_de_trayectorias, Cantidad_de_fotogramas, Matriz_tray)
        self.ArmarVideo()

In [5]:
Obj = DrawTrayectories('', 'eyeMag.avi', 'eyeMag')

In [6]:
Obj.Run(0)

('Total de trayectorias: 50979', 'Alto: 240', 'Ancho: 320', 'Cantidad de fotogramas: 160')
